In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo
import time
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Mars news URL of page to be scraped
url = "https://redplanetscience.com/"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")

# Create a empty dictionary to store the data
scraped_data = {}

### NASA Mars News

In [4]:
# Retrieve the latest news title and paragraph
news_title = soup.find_all('div', class_='content_title')[0].text
news_p = soup.find_all('div', class_='article_teaser_body')[0].text

print(news_title)
print("--------------------------------------------------------------------")
print(news_p)

NASA's Push to Save the Mars InSight Lander's Heat Probe
--------------------------------------------------------------------
The scoop on the end of the spacecraft's robotic arm will be used to 'pin' the mole against the wall of its hole.


In [6]:
# Save the scraped data to an entry of the dictionary
scraped_data["Title"] = news_title
scraped_data["Paragraph"] = news_p

### JPL Mars Space Images - Featured Image

In [7]:
# Mars Image to be scraped
jpl_nasa_url = 'https://spaceimages-mars.com'
image_url = 'https://spaceimages-mars.com/image/featured/mars2.jpg'

browser.visit(image_url)

html = browser.html

images_soup = bs(html, 'html.parser')

In [8]:
# Retrieve featured image link
relative_image_path = images_soup.find_all('img')[0]["src"]
featured_image_url = jpl_nasa_url + relative_image_path
print(featured_image_url)

https://spaceimages-mars.comhttps://spaceimages-mars.com/image/featured/mars2.jpg


In [9]:
# Save the scraped data to an entry of the dictionary
scraped_data["ImageURL"] = featured_image_url

### Mars Facts

In [10]:
# Mars facts to be scraped
facts_url = 'https://galaxyfacts-mars.com/'
tables = pd.read_html(facts_url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [11]:
#Find Mars Facts DataFrame in the lists of DataFrames
mars_facts_df = tables[1]

#Assign the columns
mars_facts_df.columns = ["Description", "Value"]
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 ( Phobos & Deimos )
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [12]:
#Save html to folder and show as html table string
mars_html_table = mars_facts_df.to_html()
mars_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millenniu

In [ ]:
mars_html_table.replace('\n', '')

'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Description</th>      <th>Value</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 ( Phobos &amp; Deimos )</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>      <th>5</th>      <td>Orbit Period:</td>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>6</th>      <td>Surface Temperature:</td>      <td>-87 to -5 °C</td>    </tr>    <tr>      <th>7</th>      <td>First Record:</td>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>8</th>      <td>Recorded By:</td>      <td>Egyptian astron

In [13]:
print(mars_html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 ( Phobos &amp; Deimos )</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
  

In [15]:
scraped_data["TableHTML"] = mars_html_table

### Mars Hemispheres

In [16]:
#Search URL
url_hemi = 'https://marshemispheres.com/'

browser.visit(url_hemi)

#HTML Object
html = browser.html

#Parse HTML with Beautiful Soup
soup = bs(html, "html.parser")

#Scrape the Mars Hemispheres Site and collect title and image urls for high-resolution images for each hemisphere
results = soup.find_all('div', class_='item')

#Create empty list
hemisphere_image_urls=[]

#Retrieve each list item
for result in results:
    title = result.find('h3').get_text()
    img_url = result.find('img', class_='thumb')['src']
    hemisphere_image_urls.append({'title': title,'img_url':img_url})
#Print Result
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'images/4e59980c1c57f89c680c0e1ccabbeff1_valles_marineris_enhanced.tif_thumb.png'}]

In [17]:
# Save the scraped data to an entry of the dictionary
scraped_data["ListImages"] = hemisphere_image_urls

In [18]:
# The scraped data is available on the dictionary form
scraped_data

{'Title': "NASA's Push to Save the Mars InSight Lander's Heat Probe",
 'Paragraph': "The scoop on the end of the spacecraft's robotic arm will be used to 'pin' the mole against the wall of its hole.",
 'ImageURL': 'https://spaceimages-mars.comhttps://spaceimages-mars.com/image/featured/mars2.jpg',
 'TableHTML': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 

##Save data to MongoDB

In [19]:
# Use flask_pymongo to set up mongo connection
conn =  "mongodb://localhost:27017/mars_mission_scraping"
client =  pymongo.MongoClient(conn)

# identify the collection and drop any existing data for this demonstration
db = client.mars_mission_scraping
db.mars_data.drop()

db.mars_data.insert_many([scraped_data])


# query_result = list(db.mars_data.find())
# query_result = (db.mars_data.find())
query_result = (db.mars_data.find_one())
query_result

{'_id': ObjectId('62eb6b5f3b956638a5320405'),
 'Title': "NASA's Push to Save the Mars InSight Lander's Heat Probe",
 'Paragraph': "The scoop on the end of the spacecraft's robotic arm will be used to 'pin' the mole against the wall of its hole.",
 'ImageURL': 'https://spaceimages-mars.comhttps://spaceimages-mars.com/image/featured/mars2.jpg',
 'TableHTML': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 ( Phobos &amp; Deimos )</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit D

In [20]:
type(query_result)

dict